In [ ]:
#ler planilha excel,pegar nome, termos banidos, preço minimo , preço maximo 
#abrir navegador, google shopping, pegar todos os resultados que tenham preço dentro da faixa e sejam o os produtos corretos
#o mesmo para o buscape
#enviar um email com uma tabela com os itens e preços encontrados junto com o link de compra

In [11]:
import pandas as pd
import os 

caminho = os.getcwd()
caminho_arquivo = caminho+ r"\buscas.xlsx"
tabela = pd.read_excel(caminho_arquivo)

display(tabela)

nome = tabela.loc[0,'Nome']
termos_banidos = tabela.loc[0,'Termos banidos']
prec_min = tabela.loc[0,'Preço mínimo']
prec_max = tabela.loc[0,'Preço máximo']


,Nome,Termos banidos,Preço mínimo,Preço máximo
0,iphone 12 64gb,mini watch,3000,3500
1,rtx 3060,zota galax,4000,4500
